# Skills Detection - Análise Exploratória

Análise das skills detectadas nas vagas de Data Engineer.

In [ ]:
import sys
sys.path.insert(0, '../../src')

import json
import pandas as pd
from collections import Counter

from skills_detection.skill_matcher import SkillMatcher, CATALOG_VERSION

## 1. Carregar Dados

In [ ]:
# Carregar catálogo de skills (JSON - mesmo formato usado em produção)
with open('../../src/skills_detection/config/skills_catalog.json') as f:
    catalog = json.load(f)

matcher = SkillMatcher.from_dict(catalog, CATALOG_VERSION)
print(f"Catálogo carregado: {len(matcher.skills)} skills")

In [ ]:
# Carregar dados do Silver (parquet)
df = pd.read_parquet('../../tmp/')
print(f"Registros: {len(df)}")
print(f"Colunas: {list(df.columns)}")

In [ ]:
df.head()

## 2. Aplicar Detecção de Skills

In [ ]:
def detect_skills_row(row):
    """Detecta skills em uma linha do DataFrame."""
    title = str(row.get('job_title') or '')
    desc = str(row.get('job_description_text') or '')
    text = f"{title} {desc}"
    return matcher.detect(text)

# Aplicar detecção
df['skills_result'] = df.apply(detect_skills_row, axis=1)

# Extrair colunas
df['skills_canonical'] = df['skills_result'].apply(lambda x: x['skills_canonical'])
df['skills_families'] = df['skills_result'].apply(lambda x: x['skills_families'])
df['skills_count'] = df['skills_canonical'].apply(len)

print(f"Skills detectadas em {len(df[df['skills_count'] > 0])} de {len(df)} vagas ({len(df[df['skills_count'] > 0])/len(df)*100:.0f}%)")

## 3. Análise de Frequência de Skills

In [ ]:
# Todas as skills encontradas
all_skills = [skill for skills in df['skills_canonical'] for skill in skills]
skill_counts = Counter(all_skills)

# Top 20 skills
top_skills = pd.DataFrame(skill_counts.most_common(20), columns=['skill', 'count'])
top_skills['percentage'] = (top_skills['count'] / len(df) * 100).round(1)
top_skills

In [ ]:
# Visualização
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(top_skills['skill'][::-1], top_skills['percentage'][::-1])
ax.set_xlabel('% das Vagas')
ax.set_title('Top 20 Skills em Vagas de Data Engineer')

# Adicionar labels
for bar, pct in zip(bars, top_skills['percentage'][::-1]):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
            f'{pct}%', va='center')

plt.tight_layout()
plt.show()

## 4. Análise por Família de Skills

In [ ]:
# Todas as famílias
all_families = [fam for families in df['skills_families'] for fam in families]
family_counts = Counter(all_families)

family_df = pd.DataFrame(family_counts.most_common(), columns=['family', 'count'])
family_df['percentage'] = (family_df['count'] / len(df) * 100).round(1)
family_df

In [ ]:
# Visualização por família
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(family_df['family'][::-1], family_df['percentage'][::-1], color='steelblue')
ax.set_xlabel('% das Vagas')
ax.set_title('Presença de Skills por Família')

for bar, pct in zip(bars, family_df['percentage'][::-1]):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
            f'{pct}%', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Top 3 skills por família
skill_family_map = {s.canonical: s.family for s in matcher.skills}

skills_with_family = []
for skills in df['skills_canonical']:
    for skill in skills:
        family = skill_family_map.get(skill, 'unknown')
        skills_with_family.append({'skill': skill, 'family': family})

skills_family_df = pd.DataFrame(skills_with_family)
top_by_family = (skills_family_df
    .groupby(['family', 'skill'])
    .size()
    .reset_index(name='count')
    .sort_values(['family', 'count'], ascending=[True, False])
    .groupby('family')
    .head(3)
)

# Pivot para visualização
for family in sorted(top_by_family['family'].unique()):
    family_data = top_by_family[top_by_family['family'] == family]
    print(f"\n{family.upper().replace('_', ' ')}:")
    for _, row in family_data.iterrows():
        pct = row['count'] / len(df) * 100
        print(f"  {row['skill']}: {row['count']} ({pct:.1f}%)")

## 5. Estatísticas por Vaga

In [ ]:
print("Estatísticas de skills por vaga:")
print(f"  Média: {df['skills_count'].mean():.1f}")
print(f"  Mediana: {df['skills_count'].median():.0f}")
print(f"  Mínimo: {df['skills_count'].min()}")
print(f"  Máximo: {df['skills_count'].max()}")
print(f"  Desvio padrão: {df['skills_count'].std():.1f}")

In [ ]:
# Distribuição de skills por vaga
fig, ax = plt.subplots(figsize=(10, 5))
df['skills_count'].hist(bins=20, ax=ax, edgecolor='black')
ax.set_xlabel('Número de Skills')
ax.set_ylabel('Número de Vagas')
ax.set_title('Distribuição de Skills por Vaga')
ax.axvline(df['skills_count'].mean(), color='red', linestyle='--', label=f"Média: {df['skills_count'].mean():.1f}")
ax.legend()
plt.show()

## 6. Co-ocorrência de Skills

In [ ]:
# Skills que aparecem juntas com frequência
from itertools import combinations

pairs = []
for skills in df['skills_canonical']:
    if len(skills) >= 2:
        for pair in combinations(sorted(skills), 2):
            pairs.append(pair)

pair_counts = Counter(pairs)
top_pairs = pd.DataFrame(pair_counts.most_common(15), columns=['pair', 'count'])
top_pairs['skill_1'] = top_pairs['pair'].apply(lambda x: x[0])
top_pairs['skill_2'] = top_pairs['pair'].apply(lambda x: x[1])
top_pairs['percentage'] = (top_pairs['count'] / len(df) * 100).round(1)
top_pairs[['skill_1', 'skill_2', 'count', 'percentage']]

In [ ]:
# Co-ocorrência de 5 skills (stacks completas)
quintuples = []
for skills in df['skills_canonical']:
    if len(skills) >= 5:
        for combo in combinations(sorted(skills), 5):
            quintuples.append(combo)

quintuple_counts = Counter(quintuples)
top_quintuples = pd.DataFrame(quintuple_counts.most_common(10), columns=['stack', 'count'])
top_quintuples['percentage'] = (top_quintuples['count'] / len(df) * 100).round(1)

print("Top 10 Stacks de 5 Skills (mais frequentes):\n")
for i, row in top_quintuples.iterrows():
    skills_str = " + ".join(row['stack'])
    print(f"{i+1}. {skills_str}")
    print(f"   Frequência: {row['count']} vagas ({row['percentage']}%)\n")

## 7. Exemplo de Vagas

In [ ]:
# Vaga com mais skills
top_vaga = df.loc[df['skills_count'].idxmax()]
print(f"Vaga com mais skills ({top_vaga['skills_count']}):")
print(f"  Título: {top_vaga['job_title']}")
print(f"  Empresa: {top_vaga.get('company_name', 'N/A')}")
print(f"  Skills: {', '.join(top_vaga['skills_canonical'][:15])}...")